# AIStore Python SDK ETL Tutorial

### Initialize ETLs


In [2]:
from aistore import Client
from aistore.client.etl_templates import MD5
import hashlib

# Note: AIS-ETLs require Kubernetes.
client = Client("http://192.168.49.2:8080")


# ETL w/ Code
def transform(input_bytes):
    md5 = hashlib.md5()
    md5.update(input_bytes)
    return md5.hexdigest().encode()


client.etl().init_code(transform=transform, etl_id="etl-code-demo")


# ETL w/ Spec
template = MD5.format(communication_type="hpush")
client.etl().init_spec(template=template, etl_id="etl-spec-demo")

### List ETLs

In [24]:
# Verify ETLs are running
client.etl().list()

[ETL(id='etl-code-demo', obj_count=0, in_bytes=0, out_bytes=0),
 ETL(id='etl-spec-demo', obj_count=0, in_bytes=0, out_bytes=0)]

### View ETLs

In [25]:
client.etl().view(etl_id="etl-code-demo")

ETLDetails(id='etl-code-demo', communication='hpush://', timeout='5m', code=b"\nimport pickle\nimport base64\n_base64code = base64.b64decode('gAWVMwIAAAAAAACMF2Nsb3VkcGlja2xlLmNsb3VkcGlja2xllIwNX2J1aWx0aW5fdHlwZZSTlIwKTGFtYmRhVHlwZZSFlFKUKGgCjAhDb2RlVHlwZZSFlFKUKEsBSwBLAEsCSwNLQ0MedACgAaEAfQF8AaACfAChAQEAfAGgA6EAoAShAFMAlE6FlCiMB2hhc2hsaWKUjANtZDWUjAZ1cGRhdGWUjAloZXhkaWdlc3SUjAZlbmNvZGWUdJSMC2lucHV0X2J5dGVzlGgMhpSMIy90bXAvaXB5a2VybmVsXzM2ODI4NC8zMzI1NTQ4MDYzLnB5lIwJdHJhbnNmb3JtlEsKQwYIAQoBDAGUKSl0lFKUfZQojAtfX3BhY2thZ2VfX5ROjAhfX25hbWVfX5SMCF9fbWFpbl9flHVOTk50lFKUjBxjbG91ZHBpY2tsZS5jbG91ZHBpY2tsZV9mYXN0lIwSX2Z1bmN0aW9uX3NldHN0YXRllJOUaB19lH2UKGgaaBSMDF9fcXVhbG5hbWVfX5RoFIwPX19hbm5vdGF0aW9uc19flH2UjA5fX2t3ZGVmYXVsdHNfX5ROjAxfX2RlZmF1bHRzX1+UTowKX19tb2R1bGVfX5RoG4wHX19kb2NfX5ROjAtfX2Nsb3N1cmVfX5ROjBdfY2xvdWRwaWNrbGVfc3VibW9kdWxlc5RdlIwLX19nbG9iYWxzX1+UfZRoC2gAjAlzdWJpbXBvcnSUk5RoC4WUUpRzdYaUhlIwLg==')\ntransform = pickle.loads(_base64code)\n\n", spec=None, dependencies='Y2xvdWRwaWNrbGU9P

In [26]:
client.etl().view(etl_id="etl-spec-demo")

ETLDetails(id='etl-spec-demo', communication='hpush://', timeout='5m', code=None, spec=b'\napiVersion: v1\nkind: Pod\nmetadata:\n  name: transformer-md5\n  annotations:\n    # Values it can take ["hpull://","hrev://","hpush://"]\n    communication_type: "hpush://"\n    wait_timeout: 5m\nspec:\n  containers:\n    - name: server\n      image: aistore/transformer_md5:latest\n      imagePullPolicy: IfNotPresent\n      ports:\n        - name: default\n          containerPort: 80\n      command: [\'/code/server.py\', \'--listen\', \'0.0.0.0\', \'--port\', \'80\']\n      readinessProbe:\n        httpGet:\n          path: /health\n          port: default\n', dependencies=None, runtime='python3')

### Get Object w/ ETL Transformation

In [27]:
# Create bucket and put object
from tests.utils import create_and_put_object

client.bucket(bck_name="bucket-demo").create()
content = create_and_put_object(
    client=client, bck_name="bucket-demo", obj_name="object-demo.jpg"
)

In [28]:
# Get object w/ ETL code transformation
client.bucket("bucket-demo").object("object-demo.jpg").get(
    etl_id="etl-code-demo"
).read_all()

b'bc9e3766f4e5de8b504fdb2e1ede7f31'

In [29]:
# Get object w/ ETL spec transformation
client.bucket("bucket-demo").object("object-demo.jpg").get(
    etl_id="etl-spec-demo"
).read_all()

b'bc9e3766f4e5de8b504fdb2e1ede7f31'

### Transform Bucket w/ ETL Transformation

In [30]:
# Create bucket to store transformed objects
client.bucket("transform-destination-bucket").create()

# Transform bucket contents (w/ on-the-fly object renames)
client.bucket("bucket-demo").transform(
    etl_id="etl-spec-demo",
    to_bck="transform-destination-bucket",
    prefix="transformed-",
    ext={"jpg": "txt"},
)

'J0KRLcLyY7'

In [31]:
# Verify rename operations for transformed objects
client.bucket("transform-destination-bucket").list_objects().get_entries()

[BucketEntry(name='transformed-object-demo.txt', size=32, checksum='fd53dfab32663975', atime='05 Aug 22 17:06 UTC', version='', target_url='', copies=0, flags=64)]

In [32]:
# Cleanup buckets
for bucket in client.cluster().list_buckets():
    client.bucket(bucket.name).delete()

### Stop ETLs

In [33]:
client.etl().stop(etl_id="etl-code-demo")
client.etl().stop(etl_id="etl-spec-demo")

### Restart Stopped ETLs

In [34]:
client.etl().start(etl_id="etl-code-demo")
client.etl().start(etl_id="etl-spec-demo")

### Stop & Delete ETLs

In [35]:
client.etl().stop(etl_id="etl-code-demo")
client.etl().stop(etl_id="etl-spec-demo")

client.etl().delete(etl_id="etl-code-demo")
client.etl().delete(etl_id="etl-spec-demo")

### Starting Deleted ETL Raises Exception

In [36]:
client.etl().start(etl_id="etl-code-demo")

AISError: STATUS:404, MESSAGE:p[KebRRGbm]: etl UUID etl-code-demo does not exist

In [37]:
client.etl().start(etl_id="etl-spec-demo")

AISError: STATUS:404, MESSAGE:p[KebRRGbm]: etl UUID etl-spec-demo does not exist

### Initialize ETL w/ before and after code and stream data

In [3]:
def before():
    return hashlib.md5()


def transform(input_bytes, md5):
    md5.update(input_bytes)


def after(md5):
    return md5.hexdigest().encode()


client.etl().init_code(
    transform=transform,
    before=before,
    after=after,
    etl_id="md5-stream",
    chunk_size=32768,
)

'md5-stream'